In [1]:
# Install the required modules
%pip install pandas numpy

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
data = pd.read_csv('house_prices_india.csv')

In [3]:
# Drop columns only if they exist in the DataFrame and data cleaning
columns_to_drop = ['area_type', 'availability', 'society']
data.drop(columns=[col for col in columns_to_drop if col in data.columns], inplace=True)

In [4]:
data.dropna(inplace=True)

In [5]:
data.location.value_counts()

location
Delhi        644
Bangalore    641
Hyderabad    641
Mumbai       636
Chennai      630
Kolkata      609
Pune         608
Ahmedabad    591
Name: count, dtype: int64

In [6]:
data['size'].unique()

array(['2 RK', '1 BHK', '4 BHK', '1 RK', '2 BHK', '3 BHK'], dtype=object)

In [7]:
data['bedrooms'] = data['size'].apply(lambda x: int(x.split(' ')[0]))

In [8]:
data.total_sqft.unique()

array([1642.41, 3473.11, 4324.26, ..., 2792.81, 4546.23, 4967.72])

In [9]:
def clean(sqft):
    tokens = sqft.split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    else:
        try:
            return float(sqft)
        except:
            return None

In [10]:
#averaging the range of sqft
clean('200-100')

150.0

In [11]:
# Apply the clean function to the original total_sqft column
data['total_sqft'] = data['total_sqft'].astype(str).apply(clean)

# Display unique values after cleaning
data['total_sqft'].unique()

array([1642.41, 3473.11, 4324.26, ..., 2792.81, 4546.23, 4967.72])

In [12]:
data.describe()

,total_sqft,bath,balcony,price,bedrooms
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2667.351196,2.501200,1.476600,253.209080,2.171600
std,1359.616784,1.128474,1.123612,142.430845,1.076655
min,301.000000,1.000000,0.000000,10.080000,1.000000
25%,1490.762500,1.000000,0.000000,128.470000,1.000000
50%,2686.410000,3.000000,1.000000,251.005000,2.000000
75%,3832.942500,4.000000,2.000000,378.600000,3.000000
max,4999.850000,4.000000,3.000000,499.970000,4.000000


In [13]:
data['sqft_per_bed'] = data['total_sqft'] / data['bedrooms']

In [14]:
data.sqft_per_bed.describe()
# Remove outliers
data2 = data[data.sqft_per_bed > 300]

In [15]:
data2['price_per_sqft'] = data2['price'] * 100000 / data2['total_sqft']

/var/folders/d6/czm8_8ns1ks66m3bz3f41ns00000gn/T/ipykernel_40606/3011775641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['price_per_sqft'] = data2['price'] * 100000 / data2['total_sqft']


In [16]:
data2.price_per_sqft.describe()

count      4609.000000
mean      12404.364059
std       13381.785296
min         250.408363
25%        4546.136774
50%        8813.144051
75%       14892.514573
max      151265.997606
Name: price_per_sqft, dtype: float64

In [17]:
data3 = data2[(data2['price_per_sqft'] >= 1000)]

In [18]:
# Drop columns only if they exist in the DataFrame
columns_to_drop = ['size', 'sqft_per_bed', 'price_per_sqft']
data3.drop(columns=[col for col in columns_to_drop if col in data3.columns], inplace=True)

/var/folders/d6/czm8_8ns1ks66m3bz3f41ns00000gn/T/ipykernel_40606/3672687338.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3.drop(columns=[col for col in columns_to_drop if col in data3.columns], inplace=True)


In [19]:
data3

,location,total_sqft,bath,balcony,price,bedrooms
0,Pune,1642.41,1,2,397.74,2
1,Kolkata,3473.11,4,0,178.08,1
2,Delhi,4324.26,1,3,186.30,4
3,Bangalore,2665.24,2,1,427.78,4
4,Ahmedabad,1332.57,3,0,150.96,1
...,...,...,...,...,...,...
4993,Kolkata,3883.88,1,3,425.22,1
4994,Hyderabad,1539.97,1,2,396.65,2
4995,Bangalore,1222.82,3,1,470.74,2
4998,Kolkata,4546.23,2,1,211.20,1


In [20]:
%pip install scikit-learn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
#one hot encoding is used to convert categorical variables into a format that can be provided to ML algorithms
#to do this we will use the OneHotEncoder class from sklearn
#standardScaler is used to standardize features by removing the mean and scaling to unit variance

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
from sklearn.model_selection import train_test_split
#train_test_split is used to split the data for training and testing dataset

In [22]:
from sklearn.linear_model import LinearRegression

In [23]:
from sklearn.pipeline import make_pipeline

In [24]:
from sklearn.compose import make_column_transformer

In [25]:
col_trans = make_column_transformer((OneHotEncoder(sparse_output=False), ['location']), remainder='passthrough')

In [26]:
lr = LinearRegression()

In [27]:
scalar = StandardScaler()

In [28]:
model = make_pipeline(col_trans, scalar, lr)

In [29]:
data_input = data3.drop(columns=['price'])
data_output = data3['price']

In [30]:
x_train, x_test, y_train, y_test = train_test_split(data_input, data_output, test_size=0.2)

In [31]:
model.fit(x_train, y_train)

/Users/shalinigavel4/Library/Python/3.9/lib/python/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [32]:
model.score(x_test, y_test)

-0.004396520749846644

In [33]:
import pandas as pd


input_data = pd.DataFrame([['Delhi', 1500, 3, 2, 3]], columns=['location', 'total_sqft', 'bath', 'balcony','bedrooms'])

In [34]:
model.predict(input_data)

array([266.53144183])

In [35]:
import pickle as pk

In [36]:
pk.dump(model, open('House_price.pkl', 'wb'))

In [37]:
data3.to_csv('cleaned_data.csv')